In [37]:
import sqlite3 as sql
import pandas as pd
from autoritetsregister_oppslag import author_gender

In [3]:
conn = sql.connect("/home/larsm/my_projects/metadata/metadata_test.db")

In [102]:
query = "select oaiid, authors from metadata_core limit 10000000 ;"

count = 1
for chunk in pd.read_sql(query, conn, chunksize=1000):
    chunk.to_csv(f"authors_{count}-{count + 1000}.csv")
    count += 1000
    if count > 2001 : break

In [129]:
res = chunk.set_index("oaiid").iloc[:5, 0].str.split("/", expand=True).applymap(author_gender, na_action='ignore')
split = res.applymap(process_result, na_action='ignore')


In [142]:
res = chunk.set_index("oaiid").iloc[:, 0].str.split("/", expand=True).applymap(author_gender, na_action='ignore')


In [143]:
split = res.applymap(process_result, na_action='ignore')

In [116]:
def process_result(lst):
    if not lst:
        return "Na"
    else:
        val = lst[0][1]
        if val == '':
            return 'Na'
        elif val in ['m', 'f']:
            return val
        else:
            raise ValueError
        

In [144]:
split1 = split.fillna('')

In [145]:
gen = split1.agg('/'.join, axis=1).str.replace('//|/$', '', regex=True).to_frame('gender')

In [137]:
gen

,gender
oaiid,
oai:nb.bibsys.no:990901257204702202,m/f/f
oai:nb.bibsys.no:990116361574702202,m
oai:nb.bibsys.no:999920000396902202,m/m/m
oai:nb.bibsys.no:999412786204702202,Na
oai:nb.bibsys.no:999919840372102202,Na


In [139]:
chunk

,oaiid,authors
0,oai:nb.bibsys.no:990901257204702202,"Bringsværd , Tor Åge / Soli , Tina / Soli , Tina"
1,oai:nb.bibsys.no:990116361574702202,"Bjørneboe , Sven Kærup"
2,oai:nb.bibsys.no:999920000396902202,"Card , Michael / Fettke , Tom E. / Isachsen , ..."
3,oai:nb.bibsys.no:999412786204702202,"Nygard , H."
4,oai:nb.bibsys.no:999919840372102202,
...,...,...
995,oai:nb.bibsys.no:999417066724702202,
996,oai:nb.bibsys.no:998221023744702202,"Sundberg , Ove Kr."
997,oai:nb.bibsys.no:999422998094702202,
998,oai:nb.bibsys.no:999215145574702202,"García Márquez , Gabriel / Risvik , Kjell"


In [148]:
chunk = chunk.join(gen, on="oaiid")

In [149]:
chunk

,oaiid,authors,gender
0,oai:nb.bibsys.no:990901257204702202,"Bringsværd , Tor Åge / Soli , Tina / Soli , Tina",m/f/f
1,oai:nb.bibsys.no:990116361574702202,"Bjørneboe , Sven Kærup",m
2,oai:nb.bibsys.no:999920000396902202,"Card , Michael / Fettke , Tom E. / Isachsen , ...",m/m/m
3,oai:nb.bibsys.no:999412786204702202,"Nygard , H.",Na
4,oai:nb.bibsys.no:999919840372102202,,Na
...,...,...,...
995,oai:nb.bibsys.no:999417066724702202,,Na
996,oai:nb.bibsys.no:998221023744702202,"Sundberg , Ove Kr.",m
997,oai:nb.bibsys.no:999422998094702202,,Na
998,oai:nb.bibsys.no:999215145574702202,"García Márquez , Gabriel / Risvik , Kjell",Na/m


In [124]:
split1

,0,1,2
oaiid,,,
oai:nb.bibsys.no:990901257204702202,m,f,f
oai:nb.bibsys.no:990116361574702202,m,,
oai:nb.bibsys.no:999920000396902202,m,m,m
oai:nb.bibsys.no:999412786204702202,NG,,
oai:nb.bibsys.no:999919840372102202,NF,,


In [57]:
author_gender("Laila Karlsen")[0][1]

'f'

In [44]:
chunk.set_index("dhlabid").iloc[:5, 0].str.split("/", expand=True)

,0,1,2
dhlabid,,,
100006626,"Bardenfleth , C.E.","Bardenfleth , J.",None
100094394,"Simonsen , Morten",None,None
100547809,"Henderson , Dee","Karlsen , Laila Dahl",None
100400316,"Michalsen , Dag","Holmøyvik , Eirik","Holmøyvik , Eirik"
100037405,Bykle,None,None


In [71]:
split

,0,1,2
dhlabid,,,
100006626,[],,None
100094394,,None,None
100547809,,[],None
100400316,,m,m
100037405,m,None,None
